# ZEC XMR Prototype

In [61]:
# Imports
import pandas as pd
from binance.client import Client
from binance.enums import * #https://github.com/sammchardy/python-binance/blob/master/binance/enums.py
import datetime
import json
import math

import schedule
import time
from datetime import datetime, timedelta

import statsmodels.formula.api as sm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from IPython.display import clear_output

def get_api_keys(site: str, api_type: str)->str:
    """
    gets api keys stored in api-keys/api-keys.txt
    site: 'binance'
    api_type: 'api', 'secret'
    """
    with open('../api-keys/api-keys.txt') as json_file:
        return json.load(json_file)[site][api_type]
# Constants
api_key = get_api_keys("binance", "api")
api_secret = get_api_keys("binance", "secret")

client = Client(api_key=api_key, api_secret=api_secret)

mode = 2

In [62]:
def get_margin_asset(name:str):
    """returns a dictionary with:
    - asset name
    - free
    - locked
    - borrowed
    - interest
    - netAsset"""
    return list(filter(lambda x: x['asset'] == name, client.get_margin_account()["userAssets"]))[0]

def binance_ceil(x:float):
    """returns the ceil to 5 decimal places (to payback borrowed amounts). Includes 0.1% trading fee"""
    return math.ceil(x*1.001*100000)/100000

def binance_floor(x:float):
    """returns the floor to 5 decimal places amount not including trading fee. (for liquidating)"""
    return math.floor(x*100000)/100000

def go_long_short(long: str, l_amt: float, short: str, s_amt: float, base="USDT"):
    """goes long and short. Enter long/short as 'XMR' rather than 'XMRUSDT'. """
    l_pair = long + base
    s_pair = short + base
    transaction = client.create_margin_loan(asset=short, amount=str(s_amt))
#     time.sleep(1)
    print("starting short order")
    order_s = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL")
    print(order_s)
    print("starting buy order")
    order_l = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL")
    print(order_l)
    send_message("ls", long=long, l_amt=str(l_amt), short=short, s_amt=str(s_amt), o_s=order_s, o_l=order_l)
    
def liquidate(long: str, short: str, base="USDT"):
    """liquidates long and short position. Enter long as currently longing which asset"""
    l_pair = long + base
    s_pair = short + base
    l_amt = binance_floor(float(get_margin_asset(long)["free"]))
    s_amt = binance_ceil(abs(float(get_margin_asset(short)["netAsset"])))
    
    order = client.create_margin_order(
        symbol=l_pair,
        side=SIDE_SELL,
        type=ORDER_TYPE_MARKET,
        quantity=l_amt,
        newOrderRespType = "FULL")
    order1 = client.create_margin_order(
        symbol=s_pair,
        side=SIDE_BUY,
        type=ORDER_TYPE_MARKET,
        quantity=s_amt,
        newOrderRespType = "FULL")
    rp = str(abs(float(get_margin_asset(short)["free"])))
    transaction1 = client.repay_margin_loan(asset=short, amount=rp)
    send_message("lq", o_s=order, o_b=order1)

    
def get_price(symbol:str):
    """returns the price. symbol MUST include USDT, ie ZECUSDT"""
    return float(client.get_recent_trades(symbol=symbol, limit=1)[0]["price"])

def trade_amt(symbol:str, total:float):
    """returns the amount of symbol to trade to be half of total. Note, symbol MUST include USDT"""
    p = get_price(symbol)
    t = total/2
    return binance_ceil(t/p)

def get_long():
    """Returns None if long nothing, else 'ZEC' or 'XMR'"""
    if float(get_margin_asset("ZEC")["borrowed"])>0: #Borrowed ZEC (ie long XMR)
        return "XMR"
    elif float(get_margin_asset("XMR")["borrowed"])>0: #Borrwed XMR (ie long ZEC)
        return "ZEC"
    else:
        return None

In [63]:
def printer():
    """main printer. Fetches current position, fetches data, and performs buys/sells if necessary"""
    long = get_long() # Returns None if long nothing, else "ZEC" or "XMR"
    
    #total will be ignored if in liquidating phase
    total = float(get_margin_asset("USDT")["free"]) 
    
    thres = 1.        
    sell_thres = 1.   
    
    # Get z_score
    z = get_z_score()
    while math.isnan(z): #z is nan, retry:
        time.sleep(1)
        z = get_z_score()
    
    # Get trading amount (won't be used if liquidating or NOT buying)
    zec = trade_amt("ZECUSDT", total)
    xmr = trade_amt("XMRUSDT", total)
    
    if z>thres and long==None: #buy/sell #LONG ZEC
        go_long_short(long="ZEC", l_amt=zec, short="XMR", s_amt=xmr)
    elif z<-thres and long==None: #buy/sell #LONG XMR
        go_long_short(long="XMR", l_amt=xmr, short="ZEC", s_amt=zec)
    elif z>sell_thres and long=="XMR": #liquidate #LONG XMR
        liquidate(long="XMR", short="ZEC")
    elif z<-sell_thres and long=="ZEC": #liquidate #LONG ZEC
        liquidate(long="ZEC", short="XMR")
    
    send_message("z", z=str(z))
    send_message("assets", usdt=get_margin_asset("USDT"), xmr=get_margin_asset("XMR"), zec=get_margin_asset("ZEC"))
    change_mode(z, long)
    reset_client()
    
    return schedule.CancelJob

In [64]:
def change_mode(z:float, long):
    """Change the frequency of the checking"""
    global mode
    if long==None:
        if z>0.5 or z<-0.5:
            mode = 2
        else:
            mode = 3
    elif long=="ZEC":
        if z > 1:
            mode = 10
        elif z > 0:
            mode = 5
        elif z > -0.5:
            mode = 3
        else:
            mode = 2
    elif long=="XMR":
        if z < -1:
            mode = 10
        elif z < 0:
            mode = 5
        elif z < 0.5:
            mode = 3
        else:
            mode = 2
    
def reset_client():
    """resets the client to prevent 'read operation timed out'"""
    global client
    global api_key
    global api_secret
    client = Client(api_key=api_key, api_secret=api_secret)

def get_z_score(): 
    '''gets the latest z-score, given hedge ratio hr.
    Warning, sometimes it gives nan, just rerun (binance's fault)'''
    zec = get_minutely_data("ZECUSDT")
    xmr = get_minutely_data("XMRUSDT")
    zec.set_index("timestamp", inplace=True)
    xmr.set_index("timestamp", inplace=True)
    
    df = pd.to_numeric(zec.open.rename("A")).to_frame()
    df["B"] = pd.to_numeric(xmr.open)
    
    df.dropna(inplace=True)
    
    results = sm.ols(formula="B ~ A", data=df[['B', 'A']]).fit()
    hr = results.params[1]
    spread = pd.Series((df['B'] - hr * df['A'])).rename("spread").to_frame()
    spread["mean"] = spread.spread.rolling(2000).mean()
    spread["std"] =  spread.spread.rolling(2000).std()
    spread["zscore"] = pd.Series((spread["spread"]-spread["mean"])/spread["std"])
    
    return spread.iloc[-1].zscore

In [65]:
def get_filtered_dataframe(df):
    """filters columns and converts columsn to floats and ints respectively"""
    df = df[['timestamp', 'open', 'high', 'low', 'close', 'volume']]
    df = df.astype(np.float64)
    df["timestamp"] = df.timestamp.astype(np.int64)
    return df

def get_minutely_data(symbol:str, days=0.5):
    """smart gets minutely data. Enter symbol with USDT"""
    data_past = pd.read_csv(f"../data/{symbol}-past.csv")

    d = datetime.today() - timedelta(days=days)
    start_date = d.strftime("%d %b %Y %H:%M:%S")
    today = datetime.today().strftime("%d %b %Y %H:%M:%S")

    klines = client.get_historical_klines(symbol, Client.KLINE_INTERVAL_1MINUTE, start_date, today, 1000)
    data = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data = get_filtered_dataframe(data)

    index = data_past.index[(data_past['timestamp'] == data.iloc[0].timestamp)].tolist()[0]
    data = pd.concat([data_past[:index], data], ignore_index=True, sort=False)

    klines = client.get_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE)
    data_latest = pd.DataFrame(klines, columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_av', 'trades', 'tb_base_av', 'tb_quote_av', 'ignore' ])
    data_latest = get_filtered_dataframe(data_latest)

    index = data.index[(data['timestamp'] == data_latest.iloc[0].timestamp)].tolist()[0]
    result = pd.concat([data[:index], data_latest], ignore_index=True, sort=False)
    result.to_csv(f"../data/{symbol}-past.csv", index=False)
    return result

In [66]:
# Discord # Saving message
def get_message():
    """gets the message"""
    with open('message.txt') as json_file:
        new_message = json.load(json_file)
        return new_message
def send_message(m:str,  **k):
    data = get_message()
    if m == "ls":
        data["message"] = f"Went long {k['l_amt']} {k['long']} at ${k['o_l']['fills'][0]['price']} \nShort {k['s_amt']} {k['short']} at ${k['o_s']['fills'][0]['price']}"
        data["o_l"] = k['o_l']
        data["o_s"] = k['o_s']
    elif m == "lq":
        o_s = k['o_s']
        o_b = k['o_b']
        data["message"] = f"Liquidated! Sold {o_s['executedQty']} {o_s['symbol']} for {o_s['fills'][0]['price']} \nBought {o_b['executedQty']} {o_b['symbol']} for {o_b['fills'][0]['price']}"
        data["o_l"] = o_s
        data["o_s"] = o_b
    elif m == "z":
        data["z"] = k["z"]
    elif m == "assets":
        data["USDT"] = k["usdt"]
        data["XMR"] = k["xmr"]
        data["ZEC"] = k["zec"]
    with open('message.txt', 'w') as outfile:
        json.dump(data, outfile)

In [67]:
schedule.clear()
# schedule.every().minute.at(":01").do(printer) #For mac
schedule.every(2).minutes.at(":01").do(printer) #For PC

Every 2 minutes at 00:00:01 do printer() (last run: [never], next run: 2021-07-15 19:23:01)

In [69]:
sleep = 1
mode = 2
while True:
    clear_output()
    print(f"Starting while loop: mode is {mode}")
    try:
        if schedule.get_jobs() != []:
            print(f"Run schedule")
            schedule.run_pending()
        else:
            print(f"Schedule is empty, setting it to run every {mode} minutes")
            schedule.every(mode).minutes.at(":01").do(printer)
    except:
        print("An error occured, sleeping for 2 minutes")
        rest = 120
        schedule.clear()
        while True:
            time.sleep(rest)
            try: 
                reset_client()
                schedule.every().minute.at(":01").do(printer)
                break
            except:
                print(f"Error again, sleeping an additional 10 seconds, to {rest + 10} seconds")
                rest += 10
    print(f"sleeping for {sleep} seconds")
    time.sleep(1)

Starting while loop: mode is 10
Run schedule
sleeping for 1 seconds


KeyboardInterrupt: 